In [1]:
from pkg.utils.utils_python3 import *
from pkg.data_collecting.load_data import *
import random
import shutil
import os

DATA_PATH = "./data"
CONVERTED_PATH = "./data/converted"
SCENE_FILENAME = "scene.pkl"
JOINT_NUM = 13
gtimer = GlobalTimer()
# ## Load Global params
DATASET_LIST = sorted(os.listdir(CONVERTED_PATH))
trainset_num = None
if trainset_num is not None:
    TRAINSET_LIST =  DATASET_LIST[:trainset_num]
    TESTSET_LIST = DATASET_LIST[trainset_num:]
else:
    TRAINSET_LIST = ['20201214-165211', '20201216-021416', '20201218-024611']
    TESTSET_LIST = ['20201208-121454', '20201212-232318', '20201213-061207']
    #TRAINSET_LIST = ['20201221-020250']
    #TESTSET_LIST = ['20201220-222433']
    
# trainset
data_loader = DataLoader(JOINT_NUM)
train_set = data_loader.get_dataset_args(TRAINSET_LIST, JOINT_NUM)
N_train = len(train_set)
test_set = data_loader.get_dataset_args(TESTSET_LIST, JOINT_NUM)
N_test = len(test_set)
print(f'Train/Test: {N_train}, {N_test}')

Train/Test: 59522, 19215


In [2]:
import tensorflow as tf

MODEL_NAME = "20210108-202908/model_23"
logpath = os.path.join('logs','gradient_tape',MODEL_NAME)
model = tf.keras.models.load_model(logpath)

@tf.function
def inference(images):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    return predictions

In [3]:
BATCH_SIZE = 12

In [4]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [5]:
#inference once

i_step = 0
scene_batch, success_batch = [], []
for scene_tuple in test_set:
    scene_data, success, skey = load_scene_data(*scene_tuple)
    i_step += 1
    scene_batch.append(scene_data)
    success_batch.append(success)
    if i_step%BATCH_SIZE==0:
        cbox, cbox_m, ccyl, ccyl_m, ibox, ibox_m, gbox, gbox_m = data_loader.separate_dat(np.array(scene_batch, dtype=np.float32))
        res_batch = inference([cbox, cbox_m, ccyl, ccyl_m, ibox, ibox_m, gbox, gbox_m])
        break

In [14]:
import SharedArray as sa
Nwdh = (15,15,15)
N_feat= data_loader.N_BEGIN_JOINT+data_loader.N_joint_label+data_loader.JOINT_NUM*3
# Create an array in shared memory.
scene_base_data_p = sa.create("shm://scene_base_data", Nwdh+(N_feat,))
cell_init_p = sa.create("shm://cell_init", (BATCH_SIZE, 3), dtype=np.int)
verts_init_p = sa.create("shm://verts_init", (BATCH_SIZE, data_loader.N_vtx_box))
chain_init_p = sa.create("shm://chain_init", (BATCH_SIZE, data_loader.JOINT_NUM), dtype=np.int)
cell_goal_p = sa.create("shm://cell_goal", (BATCH_SIZE, 3), dtype=np.int)
verts_goal_p = sa.create("shm://verts_goal", (BATCH_SIZE, data_loader.N_vtx_box))
chain_goal_p = sa.create("shm://chain_goal", (BATCH_SIZE, data_loader.JOINT_NUM), dtype=np.int)


# get data

In [19]:
scene_base_batch = np.stack([scene_base_data_p]*BATCH_SIZE)
for i_b, cell, verts, chain in zip(range(BATCH_SIZE), cell_init_p, verts_init_p, chain_init_p):
    scene_base_batch[i_b, cell[0], cell[1], cell[2], 
                     data_loader.N_BEGIN_INIT:
                     data_loader.N_BEGIN_INIT+data_loader.N_vtx_init] = verts
    scene_base_batch[i_b, cell[0], cell[1], cell[2], 
                     data_loader.N_BEGIN_INIT+data_loader.N_vtx_init:
                     data_loader.N_BEGIN_INIT+data_loader.N_vtx_init+data_loader.N_mask_init] = 1
    scene_base_batch[i_b, cell[0], cell[1], cell[2], 
                     data_loader.N_BEGIN_INIT+data_loader.N_vtx_init+data_loader.N_mask_init:
                     data_loader.N_BEGIN_INIT+data_loader.N_vtx_init+data_loader.N_mask_init+data_loader.N_joint_init] = chain
for i_b, cell, verts, chain in zip(range(BATCH_SIZE), cell_goal_p, verts_goal_p, chain_goal_p):
    scene_base_batch[i_b, cell[0], cell[1], cell[2], 
                     data_loader.N_BEGIN_GOAL:
                     data_loader.N_BEGIN_GOAL+data_loader.N_vtx_goal] = verts
    scene_base_batch[i_b, cell[0], cell[1], cell[2], 
                     data_loader.N_BEGIN_GOAL+data_loader.N_vtx_goal:
                     data_loader.N_BEGIN_GOAL+data_loader.N_vtx_goal+data_loader.N_mask_goal] = 1
    scene_base_batch[i_b, cell[0], cell[1], cell[2], 
                     data_loader.N_BEGIN_GOAL+data_loader.N_vtx_goal+data_loader.N_mask_goal:
                     data_loader.N_BEGIN_GOAL+data_loader.N_vtx_goal+data_loader.N_mask_goal+data_loader.N_joint_goal] = chain

# Inference

In [20]:
cbox, cbox_m, ccyl, ccyl_m, ibox, ibox_m, gbox, gbox_m = data_loader.separate_dat(scene_base_batch)
res_batch = inference([cbox, cbox_m, ccyl, ccyl_m, ibox, ibox_m, gbox, gbox_m])

In [21]:
res_batch

<tf.Tensor: shape=(12, 2), dtype=float32, numpy=
array([[2.9635285e-06, 9.9994445e-01],
       [2.9249775e-06, 9.9994493e-01],
       [2.8946220e-06, 9.9994516e-01],
       [2.9365290e-06, 9.9994481e-01],
       [2.8808138e-06, 9.9994528e-01],
       [2.9308635e-06, 9.9994481e-01],
       [2.8812644e-06, 9.9994528e-01],
       [2.9774803e-06, 9.9994433e-01],
       [2.8985862e-06, 9.9994516e-01],
       [2.9008233e-06, 9.9994504e-01],
       [2.9061118e-06, 9.9994504e-01],
       [2.8695554e-06, 9.9994540e-01]], dtype=float32)>

In [59]:
res_batch

<tf.Tensor: shape=(12, 2), dtype=float32, numpy=
array([[0.00818127, 0.99625754],
       [0.00820077, 0.9962573 ],
       [0.00819356, 0.9962574 ],
       [0.00819145, 0.99625754],
       [0.00825707, 0.9962502 ],
       [0.00823058, 0.99625444],
       [0.00818102, 0.9962574 ],
       [0.00821228, 0.9962572 ],
       [0.00818952, 0.9962574 ],
       [0.00821289, 0.9962572 ],
       [0.00823721, 0.9962534 ],
       [0.00819126, 0.99625754]], dtype=float32)>

In [54]:
data_loader.separate_dat(np.array(scene_batch, dtype=np.float32))

(array([[[[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
             0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        , ...,  0.        ,
             0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        , ...,  0.        ,
             0.        ,  0.        ],
           ...,
           [ 0.        ,  0.        ,  0.        , ...,  0.        ,
             0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        , ...,  0.        ,
             0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        , ...,  0.        ,
             0.        ,  0.        ]],
 
          [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
             0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        , ...,  0.        ,
             0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        , ...,  0.        ,
             0.        ,  0.        ],
         

In [13]:
sa.delete("shm://scene_base_data")
sa.delete("shm://cell_init")
sa.delete("shm://verts_init")
sa.delete("shm://chain_init")
sa.delete("shm://cell_goal")
sa.delete("shm://verts_goal")
sa.delete("shm://chain_goal")